In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [4]:
x_np = np.random.rand(100, 4).astype(np.float32)
y_np = np.random.randint(0, 2, size=(100, 1)).astype(np.int64)

x = torch.from_numpy(x_np)
y = torch.from_numpy(y_np)

In [12]:
class RedSimple(nn.Module):
    def __init__(self, n_entradas, n_ocultas, n_salidas):
        super().__init__()

        self.hidden = nn.Linear(n_entradas, n_ocultas)

        self.output = nn.Linear(n_ocultas, n_salidas)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.sigmoid(x).squeeze(1)
        return x
    
modelo = RedSimple(4, 8, 1)

In [13]:
criterion = nn.BCELoss()
optimizer = optim.Adam(modelo.parameters(), lr=0.01)

In [22]:
n_epochs = 10

batch_size = 16
dataset = torch.utils.data.TensorDataset(x, y)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(n_epochs):
    for X_batch, y_batch in dataloader:

        y_pred = modelo(X_batch)
        loss = criterion(y_pred.reshape(-1), y_batch.float().reshape(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}')
    

Epoch 1/10, Loss: 0.6848
Epoch 2/10, Loss: 0.6029
Epoch 3/10, Loss: 0.7096
Epoch 4/10, Loss: 0.6687
Epoch 5/10, Loss: 0.7452
Epoch 6/10, Loss: 0.5924
Epoch 7/10, Loss: 0.7164
Epoch 8/10, Loss: 0.7962
Epoch 9/10, Loss: 0.6561
Epoch 10/10, Loss: 0.7201


In [23]:
with torch.no_grad():
    y_pred = modelo(x)
    y_pred_class = (y_pred >= 0.5).float()
    accuracy = (y_pred_class.reshape(-1) == y.reshape(-1).float()).float().mean()
    print(f'Accuracy: {accuracy.item():.4f}')

Accuracy: 0.5500


In [24]:
print(modelo)

RedSimple(
  (hidden): Linear(in_features=4, out_features=8, bias=True)
  (output): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [25]:
import ipywidgets as widgets

capas_widget = widgets.IntSlider(value=3, min=1, max=25, step=1, description='Capas Ocultas:')

def probar_pytorch(capas):
    class RedDinamica(nn.Module):
        def __init__(self, n_entradas, n_ocultas, n_capas):
            super().__init__()

            self.layers=nn.ModuleList()
            self.layers.append(nn.Linear(n_entradas, n_ocultas))

            for _ in range(n_capas - 1):
                self.layers.append(nn.Linear(n_ocultas, n_ocultas))
            
            self.out = nn.Linear(n_ocultas, 1)
            self.relu = nn.ReLU()
            self.sig = nn.Sigmoid()
        
        def forward(self, x):
            for layer in self.layers:
                x = self.relu(layer(x))
            x = self.sig(self.out(x)).squeeze(1)
            return x
        
    model = RedDinamica(4, 8, capas)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    for _ in range(10):
        for X_batch, y_batch in dataloader:

            y_pred = model(X_batch)
            loss = criterion(y_pred.reshape(-1), y_batch.float().reshape(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    with torch.no_grad():
        y_pred = model(x)
        y_pred_class = (y_pred >= 0.5).float()
        accuracy = (y_pred_class.reshape(-1) == y.reshape(-1).float()).float().mean()
    print(f'Accuracy: {accuracy.item():.4f}')

widgets.interact(probar_pytorch, capas=capas_widget)

interactive(children=(IntSlider(value=3, description='Capas Ocultas:', max=25, min=1), Output()), _dom_classes…

<function __main__.probar_pytorch(capas)>